# Detección de Objetos con Python

## Librerías

In [1]:
import argparse
import os
import numpy as np
import json
import cv2
import copy
import imgaug as ia
from imgaug import augmenters as iaa
from keras.utils import Sequence
import xml.etree.ElementTree as ET

## Leer el Dataset

In [2]:
xml_dir = os.path.join("..", "..", "Lego_Dataset", "annotation", "lego4")
img_dir = os.path.join("..", "..", "Lego_Dataset", "images", "lego4")
labels = ['lego']
tamanio = 416
mejores_pesos = 'red_lego.h5'

In [3]:
def leer_annotations(ann_dir, img_dir, labels=[]):
    all_imgs = []
    seen_labels = []

    for ann in [x for x in sorted(os.listdir(ann_dir)) if x.endswith('.xml')]:
        img = {'object':[]}

        tree = ET.parse(ann_dir + ann)

        for elem in tree.iter():
            if 'filename' in elem.tag:
                img['filename'] = img_dir + elem.text
            if 'width' in elem.tag:
                img['width'] = int(elem.text)
            if 'height' in elem.tag:
                img['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}

                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text

                        if obj['name'] in seen_labels:
                            seen_labels[obj['name']] += 1
                        else:
                            seen_labels[obj['name']] = 1
                        
                        if len(labels) > 0 and obj['name'] not in labels:
                            break
                        else:
                            img['object'] += [obj]
                    
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text)))
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text)))
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text)))
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text)))
                            
        if len(img['object']) > 0:
            all_imgs += [img]
    
    return all_imgs, seen_labels

In [4]:
train_imgs, train_labels = leer_annotations(xml_dir, img_dir, labels)
print('Imágenes', len(train_imgs), 'Etiquetas', len(train_labels))

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\Lego_Dataset\\annotation\\lego4IMG_20200612_231240.xml'